## Train BERT model



In [18]:
test_df

,text,polarity,label
id,,,
806714101086978048,Y lo peor fue que perdí la belfie,NaN,1
806720027038728192,"usuario Ya bebí, pero soy más arrechera que pe...",NaN,1
803478166853615616,"Felicidades, usuario . Le debo un cóctel, junt...",NaN,1
785473239199850496,Aunque pensándolo bien mejor no xq sigo padeci...,NaN,1
792817362546208768,la mujer que nunca crei que me fuera a escribi...,NaN,1
...,...,...,...
756565904037670912,Un saludo para don Resentidillo muñoz.! Que si...,NaN,1
757602234041524224,"Sí, es que se nota que te importa... - Buen in...",NaN,1
786072417307328512,"usuario , usuario , usuario , usuario , estoy ...",NaN,1


In [1]:
import os
from glob import glob
import pandas as pd

def get_lang(file):
    return os.path.splitext(os.path.basename(file))[0]

def load_df(file):
    dialect = get_lang(file)
    
    df = pd.read_table(file, names=["id", "text", "polarity"], index_col=0)
    #df["dialect"] = dialect
    df["label"] = 1
    df.loc[df["polarity"] == 'P', "label"] = 2
    df.loc[df["polarity"] == 'N', "label"] = 0
    return df

train_files = glob("../data/tass2020/train/*.tsv")
dev_files = glob("../data/tass2020/dev/*.tsv")
test_files = glob("../data/tass2020/test1.1/*.tsv")

train_dfs = {get_lang(file):load_df(file) for file in train_files}
dev_dfs = {get_lang(file):load_df(file) for file in dev_files}
test_dfs = {get_lang(file):load_df(file) for file in test_files}

train_df = pd.concat(train_dfs.values())
dev_df = pd.concat(dev_dfs.values())
test_df = pd.concat(test_dfs.values())

print(len(train_df), len(dev_df), len(test_df))

train_df.columns, dev_df.columns, test_df.columns

4802 2443 7264


/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar i

(Index(['text', 'polarity', 'label'], dtype='object'),
 Index(['text', 'polarity', 'label'], dtype='object'),
 Index(['text', 'polarity', 'label'], dtype='object'))

## Preprocessing

Convertimos todos los usuarios al string "usuario"



In [2]:
import re

regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")


regex.sub("usuario", "@infobae @OnlyInPeronia jajaja me muero")


'usuario usuario jajaja me muero'

In [3]:
user_regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")
url_regex = re.compile(
    "((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)
def preprocess_tweet(text):
    """
    Basic preprocessing
    """
    text = user_regex.sub("usuario", text)
    text = url_regex.sub("url", text)
    
    return text

train_df["text"] = train_df["text"].apply(preprocess_tweet)
dev_df["text"] = dev_df["text"].apply(preprocess_tweet)
test_df["text"] = test_df["text"].apply(preprocess_tweet)


In [4]:
train_df

,text,polarity,label
id,,,
768512386269638656,usuario jajajaja... eso es verdad... aquí no h...,N,0
768529956162924544,usuario espero y deseo que el interior te caus...,NEU,1
768557093955698688,"comprendo que te molen mis tattoos, pero no te...",NEU,1
770616744192929792,"Mi última partida jugada, con Sona support. La...",P,2
769959690092642304,Tranquilos que con el.dinero de Camacho seguro...,P,2
...,...,...,...
790619641659944961,¿Qué tipo de proyecto de diseño es tu preferid...,NEU,1
785432238783619072,Que bueno ver a personas que te alegran Aunque...,P,2
776184535771906048,usuario gracias queridísimo ! Buenísimos la ch...,P,2


In [5]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

model = BertForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=3)
model = model.to(device)
model.train();



tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 80


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

Veamos primero las longitudes (a ver si no hay nada mal cargado)

In [6]:
from datasets import Dataset, Value, ClassLabel, Features

examples = pd.concat([train_df, dev_df])

features = Features({
    'text': Value('string'),
    'label': ClassLabel(num_classes=3, names=["neg", "neu", "pos"])
})

train_dataset = Dataset.from_pandas(train_df[["text", "label"]], features=features)
dev_dataset = Dataset.from_pandas(dev_df[["text", "label"]], features=features)
test_dataset = Dataset.from_pandas(test_df[["text", "label"]], features=features)

PyTorch version 1.4.0 available.


In [7]:
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['neg', 'neu', 'pos'], names_file=None, id=None)}

In [8]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32

eval_batch_size = 8

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)


Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 4802 examples in 9756046 bytes .
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 2443 examples in 4976796 bytes .
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 7264 examples in 14727658 bytes .


In [9]:
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['label']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 4802 examples in 9792126 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 2443 examples in 4991492 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 7264 examples in 14771370 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


In [10]:
from torchtext import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_it = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
dev_it = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size)
test_it = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

device

device(type='cuda')

In [11]:
from tqdm.auto import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

for epoch in range(3):
    for i, batch in enumerate(tqdm(train_it)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

loss: 1.2109912633895874
loss: 0.972237229347229
loss: 1.093443512916565
loss: 1.0072860717773438
loss: 1.0705382823944092
loss: 0.8907949924468994
loss: 0.7116267085075378
loss: 0.6812994480133057
loss: 0.8317011594772339
loss: 0.668432354927063
loss: 0.8386841416358948
loss: 0.8728552460670471
loss: 0.9339146614074707
loss: 0.752903401851654
loss: 0.6754715442657471
loss: 1.007270336151123



loss: 0.98707515001297
loss: 0.7020530700683594
loss: 0.7312524318695068
loss: 0.7171093225479126
loss: 0.8036036491394043
loss: 0.601500391960144
loss: 0.48940813541412354
loss: 0.4454194903373718
loss: 0.5841567516326904
loss: 0.48332005739212036
loss: 0.6870152354240417
loss: 0.8764623999595642
loss: 0.7250308394432068
loss: 0.616169810295105
loss: 0.5553590059280396
loss: 0.12610983848571777



loss: 0.7408760786056519
loss: 0.7059192657470703
loss: 0.5609591007232666
loss: 0.5387272834777832
loss: 0.6137959361076355
loss: 0.47368186712265015
loss: 0.4219736158847809
loss: 0.30704057216644287
loss: 0.48240798711776733
loss: 0.42426031827926636
loss: 0.6123078465461731
loss: 0.763308048248291
loss: 0.5622462034225464
loss: 0.5007971525192261
loss: 0.47867125272750854
loss: 0.04528045654296875



In [19]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support




model.eval()
preds = []
true = []
with torch.no_grad():
    for batch in tqdm(dev_it):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        outs = torch.softmax(outputs.logits, dim=1)
        true.append(batch['labels'].cpu())
        preds.append(outs.cpu())

In [27]:
predictions = torch.cat(preds).argmax(1)
labels = torch.cat(true)

In [28]:
def compute_metrics(labels, preds):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

compute_metrics(labels, predictions)


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].